In [1]:
import numpy as np

In [2]:
from __future__ import division
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
% matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from IPython.display import display
tokenizer = RegexpTokenizer(r'\w+')
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LoRe
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# nltk.download()

In [ ]:
df = pd.read_csv('../data/Spooky/train.csv')

In [ ]:
df.head()

In [ ]:
df.text = df.text.apply(lambda x: ' '.join(tokenizer.tokenize(x.lower())))

In [ ]:
df['length']=df.text.apply(lambda x:len(x.split(' ')))
df['unique_l']=df.text.apply(lambda x:len(set(x.split(' '))))
df['diversity'] = df.unique_l/df.length

In [ ]:
f,ax=plt.subplots()
for auth in df.author.unique():
    df.diversity[df.author==auth].hist(alpha=0.5,label=auth,ax=ax)
    ax.legend()

In [ ]:
f,ax=plt.subplots()
for auth in df.author.unique():
    df.length[df.author==auth].hist(alpha=0.5,label=auth,ax=ax)
    ax.legend()

In [ ]:
f,ax=plt.subplots()
for auth in df.author.unique():
    df.unique_l[df.author==auth].hist(alpha=0.5,label=auth,ax=ax)
    ax.legend()

In [ ]:
def bagging(text):
    dictio={}
    for word in text.split(' '):
        if word in dictio.keys():
            dictio[word]+=1
        else:
            dictio[word]=1
    return dictio

In [ ]:
df['bagged']=df.text.apply(bagging)

In [ ]:
per_auth={}
for auth in df.author.unique():
    df_temp=df[df.author==auth]
    result={}
    for i in range(len(df_temp.bagged)):
        x=df_temp.bagged.iloc[i]
        result={ k: x.get(k, 0) + result.get(k, 0) for k in set(x) | set(result) }
    per_auth[auth]=result
        

In [ ]:
# for word in set(per_auth['EAP'])|set(per_auth['MWS'])|set(per_auth['HPL']):
#     for auth in df.author.unique():
#         try:
#             print '{}: {} has count: {}'.format(auth,word,per_auth[auth][word])
#         except KeyError:
#             print '{}: {} has count: {}'.format(auth,word,0)

In [ ]:
for word in set(per_auth['EAP'])|set(per_auth['MWS'])|set(per_auth['HPL']):
    for auth in df.author.unique():
        if word in per_auth[auth]:
            pass
        else:
            per_auth[auth][word]=0

In [ ]:
scores = {}
for word in per_auth['EAP'].keys():
    author = ''
    maxi = 0
    suma=0
    for auth in df.author.unique():
        suma+=per_auth[auth][word]
        if per_auth[auth][word]>maxi: # colisions on draws!
            author=auth
            maxi=per_auth[auth][word]
    scores[word]=(author,maxi/suma,maxi)

In [ ]:
### scores is my trained model... basic basic model

In [ ]:
for i in range(10):
    N = np.random.randint(len(df))
    text = df.text.iloc[N]
    target = df.author.iloc[N] 
    print target,': ',text
    score = {'HPL':0,'EAP':0,'MWS':0}
    for word in text.split(' '):
        if word not in scores.keys():
            pass
        else:
            Key,Sc,L=scores[word]
            score[Key]+=Sc
    tot=0
    for k in score.keys():
        tot+=score[k]
    for k in score.keys():
        score[k]=score[k]/tot
#     print score

In [ ]:
predictions=[]
for X in df.iterrows():
    prediction={}
    txt=X[1].text
    ID=X[1].id
    score = {'HPL':0,'EAP':0,'MWS':0}
    for word in txt.split(' '):
        if word not in scores.keys():
            pass
        else:
            Key,Sc,L=scores[word]
            score[Key]+=Sc
    tot=0
    for k in score.keys():
        tot+=score[k]
    for k in score.keys():
        prediction[k]=score[k]/tot
    prediction['id']=ID
    predictions.append(prediction)

In [ ]:
subs=pd.DataFrame(predictions)

In [ ]:
subs = subs[['id','EAP','HPL','MWS']] 
subs.columns=['id','EAP_feat','HPL_feat','MWS_feat']
subs.head()

In [ ]:
new_df = pd.merge(df,subs,on='id')

In [ ]:
new_df.head()

In [ ]:
x=np.random.binomial(1,0.8,len(new_df))
lblbin = LabelBinarizer()
lblbin.fit(new_df.author.unique())
trans=lblbin.transform(new_df.author.values)
new_df['EAP_bin']= trans[:,0]
new_df['HPL_bin']= trans[:,1]
new_df['MWS_bin']= trans[:,2]
# data= new_df[['diversity','EAP_feat','HPL_feat','MWS_feat','EAP_bin','HPL_bin','MWS_bin']].values
# train_d=data[x==1][:,:-3]
# test_d=data[x==0][:,-3:]
# train_l=data[x==1][:,:-3]
# test_l=data[x==0][:,-3:]
data= new_df[['diversity','EAP_feat','HPL_feat','MWS_feat','author']].values
train_d=data[x==1][:,:-1]
train_l=data[x==1][:,-1]
test_d=data[x==0][:,:-1]
test_l=data[x==0][:,-1]


In [ ]:
lore = LoRe(solver='lbfgs')
lore.fit(train_d,train_l)
pred_p = lore.predict_proba(test_d)
pred = lore.predict(test_d)

In [ ]:
log_loss(test_l,pred_p)

In [ ]:
# final model
lore = LoRe()
lore.fit(X=new_df[['diversity','EAP_feat','HPL_feat','MWS_feat']].values,y=new_df.author.values)

In [ ]:
log_loss(new_df.author.values,lore.predict_proba(new_df[['diversity','EAP_feat','HPL_feat','MWS_feat']].values))

In [ ]:
smp=pd.read_csv('../data/Spooky/sample_submission.csv')
display(smp.head())
test=pd.read_csv('../data/Spooky/test.csv')
test.text = test.text.apply(lambda x: ' '.join(tokenizer.tokenize(x.lower())))
display(test.head())

In [ ]:
predictions=[]
for X in test.iterrows():
    prediction={}
    txt=X[1].text
    ID=X[1].id
    score = {'HPL':0,'EAP':0,'MWS':0}
    for word in txt.split(' '):
        if word not in scores.keys():
            pass
        else:
            Key,Sc,L=scores[word]
            score[Key]+=Sc
    tot=0
    for k in score.keys():
        tot+=score[k]
    for k in score.keys():
        prediction[k]=score[k]/tot
    prediction['id']=ID
    predictions.append(prediction)

In [ ]:
test_feat = pd.DataFrame(predictions)
test_feat = test_feat[['id','EAP','HPL','MWS']] 
test_feat.columns=['id','EAP_feat','HPL_feat','MWS_feat']
test_feat.head()

In [ ]:
test['length']=test.text.apply(lambda x:len(x.split(' ')))
test['unique_l']=test.text.apply(lambda x:len(set(x.split(' '))))
test['diversity'] = test.unique_l/df.length

In [ ]:
Test=pd.merge(test,test_feat, on='id')
Test.head()

In [ ]:
Test.diversity.max()

In [ ]:
data= Test[['diversity','EAP_feat','HPL_feat','MWS_feat']].values
pred_prob = lore.predict_proba(data)

In [ ]:
submi= pd.DataFrame(pred_prob)
submi['id'] = Test.id
submi

In [ ]:
submi.columns=['EAP','HPL','MWS','id']
submi
submi=submi[['id','EAP','HPL','MWS']] 
submi

In [ ]:
submi.to_csv('../data/Spooky/submission_3.csv ',index=False)

In [ ]:
submi

In [ ]:
new_df